In [4]:
%matplotlib inline

from time import time
from math import sqrt, floor
import numpy as np
import pandas as pd
from IPython.core.debugger import set_trace

pd.options.display.float_format = '{:.3f}'.format

import matplotlib.pyplot as plt
plt.style.use = "default"

In [5]:
def merge(df, labels):
    return labels.merge(df, left_index=True,right_index=True)

In [6]:
train = pd.read_csv("data/TrainData2.txt", sep='\t')
labels = pd.read_csv("data/TrainLabel2.txt", sep='\t')

In [7]:
train.describe()

,67,51,68,115,69,111,136,31,67.1,47.7,...,-9.17,-18.27,-1.8,-6.32,-20.88,-1.63,-6.13,-22.56,-5.53,-8.11
count,324.000,324.000,324.000,324.000,324.000,324.000,324.000,324.000,324.000,324.000,...,324.000,324.000,324.000,324.000,324.000,324.000,324.000,324.000,324.000,324.000
mean,57.994,38.343,61.448,96.117,58.065,99.160,85.710,27.364,58.855,55.812,...,-3.323,-20.010,-1.084,-4.370,-21.667,-0.978,-4.629,-18.985,-1.690,-4.217
std,11.712,14.595,15.022,11.088,10.654,9.430,16.433,7.454,8.887,12.358,...,8.929,2.911,1.055,1.851,3.600,0.306,1.052,3.306,0.997,1.131
min,31.000,23.000,47.000,69.000,43.000,83.000,42.000,19.000,45.000,4.950,...,-58.280,-26.790,-5.510,-10.120,-34.630,-1.830,-7.970,-29.340,-6.500,-8.930
25%,50.000,28.000,52.000,88.750,51.000,93.000,73.000,24.000,54.000,48.498,...,-4.645,-22.253,-1.750,-5.530,-24.230,-1.183,-5.403,-21.765,-2.360,-4.870
50%,57.000,31.500,55.000,94.500,54.000,96.000,85.000,25.000,57.000,57.605,...,-1.220,-19.960,-1.030,-4.485,-21.080,-0.990,-4.665,-18.860,-1.650,-4.140
75%,65.000,43.000,65.000,103.000,62.250,103.000,98.000,27.000,60.000,64.155,...,1.433,-18.247,-0.390,-2.768,-19.047,-0.780,-3.900,-16.765,-1.027,-3.288
max,107.000,91.000,124.000,141.000,100.000,138.000,130.000,84.000,114.000,86.080,...,9.580,-7.760,1.780,1.040,-12.070,0.260,-0.770,-8.330,2.580,-0.590


In [8]:
labels.describe()

,1
count,324.000
mean,2.080
std,1.001
min,1.000
25%,1.000
50%,2.000
75%,3.000
max,4.000


### Data Cleaning
Let's make it easier to deal with this data by making outliers into averages. We will take all values higher or lower than 3\*IQR and will replace them with the mean of that column

In [9]:
for col_name in train.columns:
    train_col = train[col_name]
    
    q1 = train_col.quantile(0.25)
    q3 = train_col.quantile(0.75)
    iqr = q3 - q1
    
    outside_iqr = (train_col < (q1 - 3 * iqr)) | (train_col > (q3 + 3 * iqr))
    new_col_mean = train_col[~outside_iqr].mean()
    
    train[col_name][outside_iqr] = new_col_mean

/Users/luis/.pyenv/versions/3.4.3/lib/python3.4/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()
/Users/luis/.pyenv/versions/3.4.3/lib/python3.4/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()
/Users/luis/.pyenv/versions/3.4.3/lib/python3.4/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#inde

In [10]:
train.describe()

,67,51,68,115,69,111,136,31,67.1,47.7,...,-9.17,-18.27,-1.8,-6.32,-20.88,-1.63,-6.13,-22.56,-5.53,-8.11
count,324.000,324.000,324.000,324.000,324.000,324.000,324.000,324.000,324.000,324.000,...,324.000,324.000,324.000,324.000,324.000,324.000,324.000,324.000,324.000,324.000
mean,57.994,38.019,60.006,96.117,57.935,99.040,85.710,25.364,57.508,55.812,...,-1.907,-20.010,-1.084,-4.370,-21.667,-0.978,-4.629,-18.985,-1.675,-4.217
std,11.712,14.002,12.308,11.088,10.395,9.179,16.433,2.389,5.505,12.358,...,5.440,2.911,1.055,1.851,3.600,0.306,1.052,3.306,0.960,1.131
min,31.000,23.000,47.000,69.000,43.000,83.000,42.000,19.000,45.000,4.950,...,-22.780,-26.790,-5.510,-10.120,-34.630,-1.830,-7.970,-29.340,-6.110,-8.930
25%,50.000,28.000,52.000,88.750,51.000,93.000,73.000,24.000,54.000,48.498,...,-3.510,-22.253,-1.750,-5.530,-24.230,-1.183,-5.403,-21.765,-2.337,-4.870
50%,57.000,31.500,55.000,94.500,54.000,96.000,85.000,25.000,57.000,57.605,...,-1.220,-19.960,-1.030,-4.485,-21.080,-0.990,-4.665,-18.860,-1.650,-4.140
75%,65.000,43.000,64.000,103.000,62.000,103.000,98.000,26.000,59.000,64.155,...,1.433,-18.247,-0.390,-2.768,-19.047,-0.780,-3.900,-16.765,-1.027,-3.288
max,107.000,84.000,103.000,141.000,94.000,131.000,130.000,36.000,78.000,86.080,...,9.580,-7.760,1.780,1.040,-12.070,0.260,-0.770,-8.330,2.580,-0.590


In [86]:
from sklearn.model_selection import train_test_split
X, y = train, labels.iloc[:,0]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=21)

### Random Forest Classifier
Below we try out the Random Forest Classifier provided by ScikitLearn to see how well it performs with this dataset

In [11]:
from sklearn.ensemble import RandomForestClassifier

In [87]:
clf = RandomForestClassifier(max_depth=2, random_state=0)
clf.fit(X_train, y_train)
clf.score(X_test, y_test)

0.78787878787878785

In [88]:
best_depth = 2
best_score = 0
for current_depth in range(2, 100):
    clf = RandomForestClassifier(max_depth=current_depth, random_state=0)
    clf.fit(X_train, y_train)
    current_score = clf.score(X_test, y_test)
    
    if current_score > best_score:
        best_score = current_score
        best_depth = current_depth
        
print('Best depth was {} with a score of {}'.format(best_depth, best_score))

Best depth was 8 with a score of 0.9393939393939394


### Results
As we can see, given the 90/10 split with a `random_state = 21`, the best accuracy we got from the **Random Forest Classifier** was **93.94%** with a `max_depth = 8` and `random_state = 0`